In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Edge(executable_path="./utils/msedgedriver.exe")

### Functions:

In [ ]:
def close_window() -> None:
    button = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [ ]:
def choose_category(category:str, driver:object) -> None:
    driver.find_element(by=By.XPATH, value=category).click()

In [ ]:
def exit_ad() -> None:
    ...

In [ ]:
def extract_description() -> str:
    description:str = ""
    exit_ad()
    return description

### Procesing:

In [ ]:
elements = driver.find_elements(by=By.XPATH, value="//div[@class='ann-subcat-listing']")
elements

In [ ]:
prices_obs = driver.find_elements(by=By.XPATH, value="//div[@class='ann-ad-tile__price']")
prices_obs

## Entry point